# Load data

In [ ]:
import re
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as K
import keras_tuner as kt

from utils import (
    normalize_mid_points,
    read_data
)

In [ ]:
keypoints = os.listdir('../data/keypoints')

# punch types: hook, jab, uper
keypoints_by_punch_types = [
    [i for i in keypoints if 'hook' in i],
    [i for i in keypoints if 'jab' in i],
    [i for i in keypoints if 'uper' in i]
]

In [ ]:
X_train_list = []
y_train_list = []
X_val_list = []
y_val_list = []

for keypoints in keypoints_by_punch_types:
    test_keypoints = random.choice(keypoints)

    for label in [i for i in keypoints if i != test_keypoints]:
        X, y = read_data(label, skip_midpoints=False)
        X_train_list.append(X)
        y_train_list.append(y)
    
    X, y = read_data(test_keypoints, skip_midpoints=False)
    X_val_list.append(X)
    y_val_list.append(y)


X_train = np.concatenate(X_train_list)
y_train = np.concatenate(y_train_list)
X_val = np.concatenate(X_val_list)
y_val = np.concatenate(y_val_list)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
time_steps=30 # expect camera 30 fps, so process 1 sample per second

# Number of samples in batch
N_train = X_train.shape[0] // time_steps 
N_val = X_val.shape[0] // time_steps

X_train = X_train[:N_train*time_steps].reshape(-1, time_steps, 36)
y_train = y_train[:N_train*time_steps].reshape(-1, time_steps, 1)
X_val = X_val[:N_val*time_steps].reshape(-1, time_steps, 36)
y_val = y_val[:N_val*time_steps].reshape(-1, time_steps, 1)
X_train.shape, X_val.shape

## CNN

In [ ]:
y_train = K.utils.to_categorical(y_train, num_classes=7)
X_train = tf.constant(X_train)
y_val = K.utils.to_categorical(y_val, num_classes=7)
X_val = tf.constant(X_val)

In [ ]:
X_dev = tf.expand_dims(X_train[:32], -1)
y_dev = y_train[:32]

X_dev.shape, y_dev.shape

In [ ]:
def model_builder(hp):
    # Hyperparams
    hp_pad = hp.Int('padding', min_value=1, max_value=4)
    hp_filters1 = hp.Int('filters1', min_value=16, max_value=128, step=8)
    hp_filters2 = hp.Int('filters2', min_value=16, max_value=128, step=8)
    hp_hidden_units = hp.Int('units', min_value=16, max_value=128, step=8)

    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)

    pad_1 = tf.constant([[0,0], [hp_pad, hp_pad], [0,0], [0,0]])
    
    inputs = K.Input(shape=(30, 36, 1))
    # x = tf.keras.layers.LayerNormalization(axis=-2)(inputs)
    x = inputs
    x = tf.pad(x, pad_1, 'CONSTANT')
    x = K.layers.Conv2D(filters=hp_filters1, kernel_size=(1+hp_pad*2, 36))(x)
    x = tf.squeeze(x, axis=-2)
    
    pad_2 = tf.constant([[0,0], [1,1], [0,0]])
    x = tf.pad(x, pad_2, 'CONSTANT')
    x = K.layers.Conv1D(filters=hp_filters2, kernel_size=3)(x)
    
    x = K.layers.Dense(hp_hidden_units, activation='relu')(x)
    
    out = K.layers.Dense(7, activation='softmax')(x)

    model = K.Model(inputs=inputs, outputs=out, name="cnn_model")

    model.compile(optimizer=K.optimizers.Adam(hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     directory='kt_dir2',
                     project_name='punch_dl_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

tuner.search(X_train, y_train, batch_size=4, validation_data=(X_val, y_val), 
             epochs=32, callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_hps.values

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 64 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, batch_size=1, 
                    validation_data=(X_val, y_val), 
                    epochs=64,
                   callbacks=[K.callbacks.ReduceLROnPlateau()])

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(221)
plt.title("Train Loss")
plt.plot(history.history['loss'])
plt.subplot(222)
plt.title("Train/validation accuracy")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
model.save('../models/CNN-Normalized-Stream-01')